# Earthquake localization using progressive search grid

Using pytorch GPU-accelerated engine

In [71]:
!pip install python-dateutil geopy

  Running setup.py bdist_wheel for geographiclib: started
  Running setup.py bdist_wheel for geographiclib: finished with status 'done'
  Stored in directory: C:\Users\ceefour\AppData\Local\pip\Cache\wheels\99\45\d1\14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib


In [1]:
from datetime import datetime
import dateutil.parser

# panyaungan2018
eq_lat = -7.092
eq_lon = 105.963
eq_epicenter = (eq_lat, eq_lon)
eq_depth = 48.2
eq_mw = 6.0
eq_origin_time = dateutil.parser.parse('2018-01-23T06:34:54.000+00:00')

print('EQ origin_time=%s' % (eq_origin_time))


EQ origin_time=2018-01-23 06:34:54+00:00


In [7]:
import pandas as pd

# arrivals = pd.read_csv('panyaungan2018_real.csv')
arrivals = pd.read_csv('panyaungan2018_augmented.csv')
arrivals

,station,net,lat,lon,ele,arrivalTime,epiDistance,azimuth,epiDistanceKm,waveformAvailable
0,CGJI,IA,-6.6100,105.6900,182.00,2018-01-23T06:35:03.254+00:00,0.62,-21.26,NaN,F
1,SKJI,IA,-7.0100,106.5600,100.35,2018-01-23T06:35:03.851+00:00,0.66,73.30,NaN,F
2,DBJI,IA,-6.5500,106.7500,210.84,2018-01-23T06:35:09.672+00:00,1.05,52.18,NaN,F
3,SBJI,IA,-6.1100,106.1300,64.48,2018-01-23T06:35:10.418+00:00,1.10,11.09,NaN,NaN
4,CBJI,IA,-6.7000,106.9300,1006.30,2018-01-23T06:35:10.866+00:00,1.13,63.81,NaN,F
5,CNJI,IA,-7.3100,107.1300,589.00,2018-01-23T06:35:12.060+00:00,1.21,95.46,NaN,F
6,TNGI,IA,-6.1700,106.6500,75.00,2018-01-23T06:35:12.657+00:00,1.25,35.31,NaN,F
7,TNG,IA,-6.1700,106.6500,51.25,2018-01-23T06:35:12.657+00:00,1.25,35.30,NaN,F
8,RBSI,IA,-5.8400,105.7400,224.44,2018-01-23T06:35:14.299+00:00,1.36,-7.39,NaN,F
9,BLSI,IA,-5.3700,105.2500,146.79,2018-01-23T06:35:23.104+00:00,1.95,-20.13,NaN,F


In [8]:
# Parse arrival time
# arrivals['arrivalTime_dt'] = arrivals.apply(lambda row: dateutil.parser.parse(row['arrivalTime']), axis=1)
arrivals['arrivalTime_ts'] = arrivals.apply(lambda row: dateutil.parser.parse(row['arrivalTime']).timestamp(), axis=1)
arrivals['latency'] = arrivals['arrivalTime_ts'] - eq_origin_time.timestamp()
arrivals

,station,net,lat,lon,ele,arrivalTime,epiDistance,azimuth,epiDistanceKm,waveformAvailable,arrivalTime_ts,latency
0,CGJI,IA,-6.6100,105.6900,182.00,2018-01-23T06:35:03.254+00:00,0.62,-21.26,NaN,F,1.516689e+09,9.254
1,SKJI,IA,-7.0100,106.5600,100.35,2018-01-23T06:35:03.851+00:00,0.66,73.30,NaN,F,1.516689e+09,9.851
2,DBJI,IA,-6.5500,106.7500,210.84,2018-01-23T06:35:09.672+00:00,1.05,52.18,NaN,F,1.516689e+09,15.672
3,SBJI,IA,-6.1100,106.1300,64.48,2018-01-23T06:35:10.418+00:00,1.10,11.09,NaN,NaN,1.516689e+09,16.418
4,CBJI,IA,-6.7000,106.9300,1006.30,2018-01-23T06:35:10.866+00:00,1.13,63.81,NaN,F,1.516689e+09,16.866
5,CNJI,IA,-7.3100,107.1300,589.00,2018-01-23T06:35:12.060+00:00,1.21,95.46,NaN,F,1.516689e+09,18.060
6,TNGI,IA,-6.1700,106.6500,75.00,2018-01-23T06:35:12.657+00:00,1.25,35.31,NaN,F,1.516689e+09,18.657
7,TNG,IA,-6.1700,106.6500,51.25,2018-01-23T06:35:12.657+00:00,1.25,35.30,NaN,F,1.516689e+09,18.657
8,RBSI,IA,-5.8400,105.7400,224.44,2018-01-23T06:35:14.299+00:00,1.36,-7.39,NaN,F,1.516689e+09,20.299
9,BLSI,IA,-5.3700,105.2500,146.79,2018-01-23T06:35:23.104+00:00,1.95,-20.13,NaN,F,1.516689e+09,29.104


In [9]:
# Sort based on earliest arrival time
arrivals_sorted = arrivals.sort_values('arrivalTime_ts')
arrivals_sorted

,station,net,lat,lon,ele,arrivalTime,epiDistance,azimuth,epiDistanceKm,waveformAvailable,arrivalTime_ts,latency
0,CGJI,IA,-6.6100,105.6900,182.00,2018-01-23T06:35:03.254+00:00,0.62,-21.26,NaN,F,1.516689e+09,9.254
1,SKJI,IA,-7.0100,106.5600,100.35,2018-01-23T06:35:03.851+00:00,0.66,73.30,NaN,F,1.516689e+09,9.851
2,DBJI,IA,-6.5500,106.7500,210.84,2018-01-23T06:35:09.672+00:00,1.05,52.18,NaN,F,1.516689e+09,15.672
3,SBJI,IA,-6.1100,106.1300,64.48,2018-01-23T06:35:10.418+00:00,1.10,11.09,NaN,NaN,1.516689e+09,16.418
4,CBJI,IA,-6.7000,106.9300,1006.30,2018-01-23T06:35:10.866+00:00,1.13,63.81,NaN,F,1.516689e+09,16.866
5,CNJI,IA,-7.3100,107.1300,589.00,2018-01-23T06:35:12.060+00:00,1.21,95.46,NaN,F,1.516689e+09,18.060
6,TNGI,IA,-6.1700,106.6500,75.00,2018-01-23T06:35:12.657+00:00,1.25,35.31,NaN,F,1.516689e+09,18.657
7,TNG,IA,-6.1700,106.6500,51.25,2018-01-23T06:35:12.657+00:00,1.25,35.30,NaN,F,1.516689e+09,18.657
8,RBSI,IA,-5.8400,105.7400,224.44,2018-01-23T06:35:14.299+00:00,1.36,-7.39,NaN,F,1.516689e+09,20.299
10,BBJI,GE,-7.4626,107.6500,615.00,2018-01-23T06:35:19.777+00:00,1.74,98.94,190.6,T,1.516689e+09,25.777


In [11]:
# Cut only 3 nearest stations
arrivals_sorted = arrivals_sorted[:3]
arrivals_sorted

,station,net,lat,lon,ele,arrivalTime,epiDistance,azimuth,epiDistanceKm,waveformAvailable,arrivalTime_ts,latency
0,CGJI,IA,-6.61,105.69,182.00,2018-01-23T06:35:03.254+00:00,0.62,-21.26,NaN,F,1.516689e+09,9.254
1,SKJI,IA,-7.01,106.56,100.35,2018-01-23T06:35:03.851+00:00,0.66,73.30,NaN,F,1.516689e+09,9.851
2,DBJI,IA,-6.55,106.75,210.84,2018-01-23T06:35:09.672+00:00,1.05,52.18,NaN,F,1.516689e+09,15.672


In [54]:
# Earliest station
p1 = arrivals_sorted.iloc[1]
p1

station                                       SKJI
net                                             IA
lat                                          -7.01
lon                                         106.56
ele                                         100.35
arrivalTime          2018-01-23T06:35:03.851+00:00
epiDistance                                   0.66
azimuth                                       73.3
epiDistanceKm                                  NaN
waveformAvailable                                F
arrivalTime_ts                         1.51669e+09
latency                                      9.851
Name: 1, dtype: object

In [13]:
# Arrival time of earliest station
t1 = p1['arrivalTime_ts']
t1

1516689303.851

In [14]:
p1_loc = (p1['lat'], p1['lon'])
p1_loc

(-7.01, 106.56)

In [15]:
# Make a grid using 3-deg radius
def round_res(num: float, resolution: float): 
    return round(num / resolution) * resolution

p1_locr = (round_res(p1_loc[0], 0.3), round_res(p1_loc[1], 0.3))
p1_locr

(-6.8999999999999995, 106.5)

In [424]:
import numpy as np
import torch
import torch.cuda

point_interval = 0.1
lats = torch.arange(p1_locr[0] - 1.2, p1_locr[0] + 1.2 + 0.001, point_interval, dtype=torch.float, device='cuda')
lons = torch.arange(p1_locr[1] - 1.2, p1_locr[1] + 1.2 + 0.001, point_interval, dtype=torch.float, device='cuda')
n_points = len(lats) * len(lons)
print('lats=%s lons=%s len=%dx%d n_points=%d' % (lats, lons, len(lats), len(lons), n_points))


lats=tensor([-8.1000, -8.0000, -7.9000, -7.8000, -7.7000, -7.6000, -7.5000, -7.4000,
        -7.3000, -7.2000, -7.1000, -7.0000, -6.9000, -6.8000, -6.7000, -6.6000,
        -6.5000, -6.4000, -6.3000, -6.2000, -6.1000, -6.0000, -5.9000, -5.8000,
        -5.7000], device='cuda:0') lons=tensor([105.3000, 105.4000, 105.5000, 105.6000, 105.7000, 105.8000, 105.9000,
        106.0000, 106.1000, 106.2000, 106.3000, 106.4000, 106.5000, 106.6000,
        106.7000, 106.8000, 106.9000, 107.0000, 107.1000, 107.2000, 107.3000,
        107.4000, 107.5000, 107.6000, 107.7000], device='cuda:0') len=25x25 n_points=625


In [425]:
grid = torch.empty([n_points, 2], dtype=torch.float, device='cuda') # our lat lon is just approximate
grid[0]

tensor([1.4013e-45, 5.2886e-14], device='cuda:0')

In [426]:
for lat_idx, lat in enumerate(lats):
    for lon_idx, lon in enumerate(lons):
        idx = lat_idx * len(lons) + lon_idx
        grid[idx, 0] = lat
        grid[idx, 1] = lon
grid[:5]

tensor([[ -8.1000, 105.3000],
        [ -8.1000, 105.4000],
        [ -8.1000, 105.5000],
        [ -8.1000, 105.6000],
        [ -8.1000, 105.7000]], device='cuda:0')

In [427]:
from datetime import timedelta

times = torch.empty(31, dtype=torch.float64, device='cuda') # MUST be 64-bit!
for idx, sec in enumerate(range(31)):
    print('%s - %s = %s' % (t1, sec, t1 - sec))
    times[idx] = t1 - sec
print(len(times))
times

1516689303.851 - 0 = 1516689303.851
1516689303.851 - 1 = 1516689302.851
1516689303.851 - 2 = 1516689301.851
1516689303.851 - 3 = 1516689300.851
1516689303.851 - 4 = 1516689299.851
1516689303.851 - 5 = 1516689298.851
1516689303.851 - 6 = 1516689297.851
1516689303.851 - 7 = 1516689296.851
1516689303.851 - 8 = 1516689295.851
1516689303.851 - 9 = 1516689294.851
1516689303.851 - 10 = 1516689293.851
1516689303.851 - 11 = 1516689292.851
1516689303.851 - 12 = 1516689291.851
1516689303.851 - 13 = 1516689290.851
1516689303.851 - 14 = 1516689289.851
1516689303.851 - 15 = 1516689288.851
1516689303.851 - 16 = 1516689287.851
1516689303.851 - 17 = 1516689286.851
1516689303.851 - 18 = 1516689285.851
1516689303.851 - 19 = 1516689284.851
1516689303.851 - 20 = 1516689283.851
1516689303.851 - 21 = 1516689282.851
1516689303.851 - 22 = 1516689281.851
1516689303.851 - 23 = 1516689280.851
1516689303.851 - 24 = 1516689279.851
1516689303.851 - 25 = 1516689278.851
1516689303.851 - 26 = 1516689277.851
1516689303.

tensor([1.5167e+09, 1.5167e+09, 1.5167e+09, 1.5167e+09, 1.5167e+09, 1.5167e+09,
        1.5167e+09, 1.5167e+09, 1.5167e+09, 1.5167e+09, 1.5167e+09, 1.5167e+09,
        1.5167e+09, 1.5167e+09, 1.5167e+09, 1.5167e+09, 1.5167e+09, 1.5167e+09,
        1.5167e+09, 1.5167e+09, 1.5167e+09, 1.5167e+09, 1.5167e+09, 1.5167e+09,
        1.5167e+09, 1.5167e+09, 1.5167e+09, 1.5167e+09, 1.5167e+09, 1.5167e+09,
        1.5167e+09], device='cuda:0', dtype=torch.float64)

In [428]:
eq_vel_deg = 0.067 # Primary wave velocity = 0.067 degree / second
eq_vel_m = 7437 # Primary wave velocity = 0.067 deg/s * 111000 = 7437 m/s

velocities = torch.empty(1, dtype=torch.float16, device='cuda')
velocities[0] = eq_vel_m
velocities

tensor([7436.], device='cuda:0', dtype=torch.float16)

## Weighted RMSE

$$\text{wRMSE} = \sqrt{\sum_{i=1}^n w_i (\hat T_i - T_i)^2}$$


In [429]:
from geopy import distance

In [430]:
# https://gist.github.com/nickjevershed/6480846
import math

def distance_on_sphere(a: torch.Tensor, b: torch.Tensor, radius: float):

    # Convert latitude and longitude to 
    # spherical coordinates in radians.
    degrees_to_radians = math.pi/180.0
        
    # phi = 90 - latitude
    phi1 = (90.0 - a.select(4, 0)) * degrees_to_radians
    phi2 = (90.0 - b.select(4, 0)) * degrees_to_radians
        
    # theta = longitude
    theta1 = a.select(4, 1) * degrees_to_radians
    theta2 = b.select(4, 1) * degrees_to_radians
        
    # Compute spherical distance from spherical coordinates.
        
    # For two locations in spherical coordinates 
    # (1, theta, phi) and (1, theta, phi)
    # cosine( arc length ) = 
    #    sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length
    
    cos = (torch.sin(phi1) * torch.sin(phi2) * torch.cos(theta1 - theta2) + 
           torch.cos(phi1) * torch.cos(phi2))
    arc = torch.acos( cos )

    # Remember to multiply arc by the radius of the earth 
    # in your favorite set of units to get length.
    return arc * radius


In [431]:
def score_stations(station_locs: torch.Tensor, actual_arrivals: torch.Tensor,
                  epicenter_es: torch.Tensor, origin_time_es: torch.Tensor, velocity_es: torch.Tensor):
    '''Calculate distance (m) and the error of arrival time (seconds) for given stations and given solution parameters (epicenter, origin time, velocity).'''
    # Returned shape is E x O x V x S x 2
    # where E=length of epicenters, O=length of origin times, V=length of velocities, S=length of stations

#     print('Dists traditional...')
#     dists = torch.empty(len(epicenter_es), len(origin_time_es), len(velocity_es), len(station_locs), 
#                         dtype=torch.float64, device='cuda')
#     for epicenter_idx, epicenter_e in enumerate(epicenter_es):
#         for station_idx, station_loc in enumerate(station_locs):
#             dist = distance.distance(epicenter_e, station_loc).m
#             for origin_time_idx, origin_time_e in enumerate(origin_time_es):
#                 for velocity_idx, velocity_e in enumerate(velocity_es):
#                     dists[epicenter_idx][origin_time_idx][velocity_idx][station_idx] = dist

#     print('Dists geodesic + expand...')
#     dists = torch.empty(len(epicenter_es), 1, 1, len(station_locs), 
#                         dtype=torch.float64, device='cuda')
# #     print('dists: %s' % (dists.size(),))
#     for epicenter_idx, epicenter_e in enumerate(epicenter_es):
#         for station_idx, station_loc in enumerate(station_locs):
#             dist = distance.distance(epicenter_e, station_loc).m
# #             dist = distance.great_circle(epicenter_e, station_loc).m
#             dists[epicenter_idx][0][0][station_idx] = dist
#     print('Dists expand...')
#     dists.expand(-1, len(origin_time_es), len(velocity_es), -1)
    
    print('Dists great circle (GPU) + expand...')
    dists1 = torch.empty(len(epicenter_es), 1, 1, len(station_locs), 2,
                        dtype=torch.float64, device='cuda')
    dists2 = torch.empty(len(epicenter_es), 1, 1, len(station_locs), 2,
                        dtype=torch.float64, device='cuda')
    for epicenter_idx, epicenter_e in enumerate(epicenter_es):
        for station_idx, station_loc in enumerate(station_locs):
            dists1[epicenter_idx][0][0][station_idx] = epicenter_e
            dists2[epicenter_idx][0][0][station_idx] = station_loc
    dists = distance_on_sphere(dists1, dists2, distance.EARTH_RADIUS * 1000)
    print('Dists expand...')
    dists.expand(-1, len(origin_time_es), len(velocity_es), -1)
        
#     print('dists = %s' % dists)
    print('Travel durations...')
    travel_durs = dists / eq_vel_m
#     print('travel_durs %s = %s' % (travel_durs.size(), travel_durs))
    print('Arrival times...')
    origin_time_x = origin_time_es.view(1, len(origin_time_es), 1, 1).expand(len(epicenter_es), -1, len(velocity_es), len(station_locs))
    arrival_times_es = origin_time_x + travel_durs
#     print('arrival_times_es = %s' % arrival_times_es)
#     print('actual_arrivals = %s' % actual_arrivals)
    print('Arrival time errors...')
    arrival_errors = arrival_times_es - actual_arrivals
#     print('arrival_errors = %s' % arrival_errors)
#     print('From epicenter %s to station %s %s (%d km) in %s s' % 
#           (epicenter_e, station_code, station_loc, dist_m/1000, travel_dur))
#     print('Est arrival %s actual %s error=%s' % 
#           (arrival_time_e, actual_arrival, arrival_error))
    return (dists, arrival_errors)

station_codes = list(arrivals_sorted['station'])
print(station_codes)
station_locs = torch.Tensor([arrivals_sorted['lat'], arrivals_sorted['lon']]).transpose(0, 1)
print(station_locs)
actual_arrivals = torch.cuda.DoubleTensor(arrivals_sorted['arrivalTime_ts'])
print(actual_arrivals)

epicenter_e = torch.tensor([-9.899999999999999, 103.5], dtype=torch.float, device='cuda')
origin_time_e = dateutil.parser.parse('2018-01-23T06:35:03.851000+00:00').timestamp()
print('epicenter_e = %s' % epicenter_e)
epicenter_es = torch.unsqueeze(epicenter_e, 0)
print('epicenter_es = %s' % epicenter_es)
origin_time_es = torch.tensor([origin_time_e], dtype=torch.double, device='cuda')
print(origin_time_es)
score_stations(station_locs, actual_arrivals, epicenter_es, origin_time_es, velocities)

['CGJI', 'SKJI', 'DBJI']
tensor([[ -6.6100, 105.6900],
        [ -7.0100, 106.5600],
        [ -6.5500, 106.7500]])
tensor([1.5167e+09, 1.5167e+09, 1.5167e+09], device='cuda:0', dtype=torch.float64)
epicenter_e = tensor([ -9.9000, 103.5000], device='cuda:0')
epicenter_es = tensor([[ -9.9000, 103.5000]], device='cuda:0')
tensor([1.5167e+09], device='cuda:0', dtype=torch.float64)
Dists great circle (GPU) + expand...
Dists expand...
Travel durations...
Arrival times...
Arrival time errors...


(tensor([[[[438057.1466, 465310.3825, 516376.7338]]]],
        device='cuda:0', dtype=torch.float64),
 tensor([[[[59.4994, 62.5669, 63.6125]]]], device='cuda:0', dtype=torch.float64))

In [432]:
# Estimated wrong one with 0.0020 RWSSE
epicenter_es = torch.unsqueeze(grid[111], 0)
origin_time_es = torch.unsqueeze(times[28], 0)
score_stations(station_locs, actual_arrivals, epicenter_es, origin_time_es, velocities)

Dists great circle (GPU) + expand...
Dists expand...
Travel durations...
Arrival times...
Arrival time errors...


(tensor([[[[144312.3204,  78727.2949, 133578.1277]]]],
        device='cuda:0', dtype=torch.float64),
 tensor([[[[ -7.9984, -17.4141, -15.8597]]]], device='cuda:0', dtype=torch.float64))

In [433]:
# Should be correct one? with 0.0020 RWSSE
grid
# epicenter_es = torch.unsqueeze(grid[111], 0)
# origin_time_es = torch.unsqueeze(times[28], 0)
#score_stations(station_locs, actual_arrivals, epicenter_es, origin_time_es, velocities)

tensor([[ -8.1000, 105.3000],
        [ -8.1000, 105.4000],
        [ -8.1000, 105.5000],
        ...,
        [ -5.7000, 107.5000],
        [ -5.7000, 107.6000],
        [ -5.7000, 107.7000]], device='cuda:0')

In [434]:
# MULTI-SOLUTION ROWS VERSION
def score_solutions(station_codes: list, station_locs: torch.Tensor, actual_arrivals: torch.Tensor,
          epicenter_es: torch.Tensor, origin_time_es: torch.Tensor, velocity_es: torch.Tensor):
    '''Returns the RWSSE for all stations given parameters containing estimates.'''
    
#     score_station(stations.iloc[0]['station'], (stations.iloc[0]['lat'], stations.iloc[0]['lon']),
#                   stations.iloc[0]['arrivalTime_dt'],
#                   epicenter_e, origin_time_e)
    (dists, arrival_errors) = score_stations(station_locs, actual_arrivals,
                             epicenter_es, origin_time_es, velocity_es)
#     print(result_arr)
    print('Square error...')
    ses = torch.pow(arrival_errors, 2)
    print('Weighted SE...')
    weighted_ses = torch.mul(1 / dists, ses)
#     weighted_ses = ses
#     print(weighted_ses)
    print('Sum...')
    wsse = weighted_ses.sum(3, dtype=torch.float16) # only need for reasonable precision here
#     print(wsse)
    print('Root...')
    rwsse = torch.sqrt(wsse)
#     print('RWSSEs: %s' % rwsse)
    return rwsse

station_codes = list(arrivals_sorted['station'])
# print(station_codes)
station_locs = torch.cuda.HalfTensor([arrivals_sorted['lat'], arrivals_sorted['lon']]).transpose(0, 1)
#print(station_locs)
actual_arrivals = torch.cuda.DoubleTensor(arrivals_sorted['arrivalTime_ts'])
# print(actual_arrivals)
# print(grid[:2])
# print(velocities)
# print(times[:15])
score_solutions(station_codes, station_locs, actual_arrivals, grid[:2], times[:2], velocities)

Dists great circle (GPU) + expand...
Dists expand...
Travel durations...
Arrival times...
Arrival time errors...
Square error...
Weighted SE...
Sum...
Root...


tensor([[[0.0970],
         [0.0931]],

        [[0.0954],
         [0.0914]]], device='cuda:0', dtype=torch.float16)

In [443]:
# Let's score everything!
import time

print('Scoring %d x %d x %d = %d combinations...' % 
      (len(grid), len(times), len(velocities), len(grid) * len(times) * len(velocities)))
start = time.process_time()
scores = score_solutions(station_codes, station_locs, actual_arrivals, grid, times, velocities)
elapsed = time.process_time() - start
print('Scored in %s seconds' % elapsed)
print(scores.size())

Scoring 625 x 31 x 1 = 19375 combinations...
Dists great circle (GPU) + expand...
Dists expand...
Travel durations...
Arrival times...
Arrival time errors...
Square error...
Weighted SE...
Sum...
Root...
Scored in 0.421875 seconds
torch.Size([625, 31, 1])


In [444]:
scores

tensor([[[0.0970],
         [0.0931],
         [0.0892],
         ...,
         [0.0137],
         [0.0176],
         [0.0215]],

        [[0.0954],
         [0.0914],
         [0.0874],
         ...,
         [0.0168],
         [0.0208],
         [0.0247]],

        [[0.0938],
         [0.0898],
         [0.0857],
         ...,
         [0.0199],
         [0.0239],
         [0.0280]],

        ...,

        [[0.0918],
         [0.0880],
         [0.0840],
         ...,
         [0.0489],
         [0.0519],
         [0.0551]],

        [[0.0941],
         [0.0903],
         [0.0864],
         ...,
         [0.0449],
         [0.0476],
         [0.0506]],

        [[0.0964],
         [0.0926],
         [0.0889],
         ...,
         [0.0412],
         [0.0436],
         [0.0462]]], device='cuda:0', dtype=torch.float16)

In [445]:
best_score = torch.min(scores)
best_score

tensor(0.0008, device='cuda:0', dtype=torch.float16)

In [446]:
best_idxs = (scores == best_score).nonzero()
best_idx = best_idxs[0]
best_idx

tensor([231,  10,   0], device='cuda:0')

In [447]:
times[28].item()

1516689275.851

In [448]:
(epicenter_e_idx, origin_time_e_idx, velocity_e_idx) = best_idx
epicenter_e_idx

tensor(231, device='cuda:0')

In [449]:
from datetime import timezone

epicenter_e = grid[epicenter_e_idx]
origin_time_e = datetime(1970, 1, 1, tzinfo=timezone.utc) + timedelta(seconds=times[origin_time_e_idx].item())
velocity_e = velocities[velocity_e_idx]

In [450]:
t_last_offset = np.round((arrivals_sorted.tail(1)['arrivalTime_ts'].item() - eq_origin_time.timestamp()) / 0.001) * 0.001
epicenter_err = distance.distance(epicenter_e, eq_epicenter).km
origin_time_err = abs((origin_time_e - eq_origin_time).total_seconds())
print('Estimated epicenter: %s. Actual epicenter: %s.' % (np.array(epicenter_e), eq_epicenter))
print('Estimated origin time: %s. Actual origin time: %s.' % (origin_time_e, eq_origin_time))
print('Estimated P-wave velocity: %s m/s' % velocity_e.item())
print('Decision latency: T + %s seconds. Epicenter error: ±%d km. Origin time error: ±%s seconds.' %
      (t_last_offset, epicenter_err, origin_time_err))

Estimated epicenter: [ -7.2000003 105.9      ]. Actual epicenter: (-7.092, 105.963).
Estimated origin time: 2018-01-23 06:34:53.851000+00:00. Actual origin time: 2018-01-23 06:34:54+00:00.
Estimated P-wave velocity: 7436.0 m/s
Decision latency: T + 15.672 seconds. Epicenter error: ±13 km. Origin time error: ±0.149 seconds.
